In [20]:
!pip install azure-cosmos pandas pyarrow

In [21]:
import io
import json
import requests
import pandas as pd


def tba_df(url):
  """
  Retrieves a URL from The Blue Alliance and converts it to a dataframe
  """
  tba = 'https://www.thebluealliance.com/api/v3'
  # We must specify this extra HTTP header so TBA knows what account is
  # accessing the data.
  headers = {'X-TBA-Auth-Key': 'VUdGT41tnb2MLMfXSX0QvPSDDyi0aMVzmxLTWar8wjCmZvNFZfZiq7sHfhAtWgbO'}
  full_url = f'{tba}{url}'
  # Uncomment this to see the URL you'r trying to contact.  Sometimes a simple
  # mistake like having two slashes (//) instead of one (/) means the server
  # can't process your request.
  # print(f'Retrieving {full_url}')
  r = requests.get(full_url, headers=headers)
  json_data = r.text
  print(json_data)
  obj = json.loads(json_data)
  print(obj)
  df = pd.read_json(io.StringIO(json_data))
  return df 

def get_team_events_df(team_key, year):
    """
    Parameters:
    -----------
        team_key: str
            TBA key for a team. Example: 'frc4003'
        year: str
            Year of events we're looking for
    Returns:
    --------
    DataFrame
        DataFrame contains information on every event a team participated in
        for the given year.
    """
    df = tba_df(f'/team/{team_key}/events/{year}')
    return df


In [22]:
orig_matches = tba_df(f'/event/2023miwmi/matches')
"""
gvsu = tba_df(f'/event/2023miwmi/teams')
matches = None
for teamkey in gvsu['key']:
    print(teamkey)
    tmatches = tba_df(f'/team/{teamkey}/matches/2023')
    tmatches['team_key'] = teamkey
    matches = tmatches if matches is None else pd.concat([matches, tmatches])
"""

[
  {
    "actual_time": 1679669532,
    "alliances": {
      "blue": {
        "dq_team_keys": [],
        "score": 135,
        "surrogate_team_keys": [],
        "team_keys": [
          "frc70",
          "frc5927",
          "frc74"
        ]
      },
      "red": {
        "dq_team_keys": [],
        "score": 119,
        "surrogate_team_keys": [],
        "team_keys": [
          "frc7248",
          "frc4967",
          "frc1025"
        ]
      }
    },
    "comp_level": "qm",
    "event_key": "2023miwmi",
    "key": "2023miwmi_qm1",
    "match_number": 1,
    "post_result_time": 1679669708,
    "predicted_time": 1679670000,
    "score_breakdown": {
      "blue": {
        "activationBonusAchieved": true,
        "adjustPoints": 0,
        "autoBridgeState": "Level",
        "autoChargeStationPoints": 12,
        "autoChargeStationRobot1": "Docked",
        "autoChargeStationRobot2": "None",
        "autoChargeStationRobot3": "None",
        "autoCommunity": {
          "B": [

/home/jbuist/git/FRCScout/.venv/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/home/jbuist/git/FRCScout/.venv/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


"\ngvsu = tba_df(f'/event/2023miwmi/teams')\nmatches = None\nfor teamkey in gvsu['key']:\n    print(teamkey)\n    tmatches = tba_df(f'/team/{teamkey}/matches/2023')\n    tmatches['team_key'] = teamkey\n    matches = tmatches if matches is None else pd.concat([matches, tmatches])\n"

In [23]:
matches = orig_matches.copy()
sb = pd.DataFrame(pd.json_normalize(matches.score_breakdown))
alliances = pd.DataFrame(pd.json_normalize(matches.alliances))
matches = matches.merge(sb, left_index=True, right_index=True)
matches = matches.merge(alliances, left_index=True, right_index=True)
matches.drop(columns=['score_breakdown'], inplace=True)
matches['blue_teams'] = matches['blue.team_keys'].apply(lambda x: f'{x[0]},{x[1]},{x[2]}')
matches['red_teams'] = matches['red.team_keys'].apply(lambda x: f'{x[0]},{x[1]},{x[2]}')
matches['team_keys'] = matches['blue_teams'] + ',' + matches['red_teams']
matches['team_keys']
teams = matches['team_keys'].str.split(',').apply(pd.Series, 1).stack()
teams.index = teams.index.droplevel(-1)
teams.name = 'team_key'
matches = matches.join(teams)

In [24]:
def robot_number(row):
    alliance = row.alliance
    num = row.alliances[alliance]['team_keys'].index(row.team_key)
    return num+1

def auto_community(row):
    alliance = row.alliance
    num = row.robot_number
    colname = f'{alliance}.mobilityRobot{num}'
    if row.get(colname) == 'Yes':
        return 1
    return 0

def auto_docked(row):
    alliance = row.alliance
    num = row.robot_number
    colname = f'{alliance}.autoChargeStationRobot{num}'
    if row.get(colname) == 'Docked':
        return 1
    return 0

def auto_docked_level(row):
    state = row.get(f'{row.alliance}.autoBridgeState')
    if state == 'Level' and row.auto_docked == 1:
        return 1
    return 0

def auto_gp_points(row):
    alliance = row.alliance
    autoGP = row.get(f'{alliance}.autoGamePiecePoints')
    return autoGP

def auto_points(row):
    alliance = row.alliance
    autoP = row.get(f'{alliance}.autoPoints')
    return autoP

def tele_gp_points(row):
    alliance = row.alliance
    teleGP = row.get(f'{alliance}.teleopGamePiecePoints')
    return teleGP

def tele_points(row):
    alliance = row.alliance
    teleP = row.get(f'{alliance}.teleopPoints')
    return teleP

def endgame_parked(row):
    alliance = row.alliance
    num = row.robot_number
    colname = f'{alliance}.endGameChargeStationRobot{num}'
    if row.get(colname) == 'Park':
        return 1
    return 0

def endgame_docked(row):
    alliance = row.alliance
    num = row.robot_number
    colname = f'{alliance}.endGameChargeStationRobot{num}'
    if row.get(colname) == 'Docked':
        return 1
    return 0

def endgame_docked_level(row):
    state = row.get(f'{row.alliance}.endGameBridgeState')
    if state == 'Level' and row.endgame_docked == 1:
        return 1
    return 0

def endgame_points(row):
    alliance = row.alliance
    endgameP = (
        row.get(f'{alliance}.endGameParkPoints') +
        row.get(f'{alliance}.endGameChargeStationPoints')
    )
    return endgameP 


def link_points(row):
    alliance = row.alliance
    endgameP = row.get(f'{alliance}.linkPoints')
    return endgameP 

def rank_points(row):
    alliance = row.alliance
    endgameP = row.get(f'{alliance}.rp')
    return endgameP 

def total_points(row):
    alliance = row.alliance
    endgameP = row.get(f'{alliance}.totalPoints')
    return endgameP 


p = matches.copy()
p['alliance'] = p.apply(lambda r: 'blue' if r.team_key in r.alliances['blue']['team_keys'] else 'red', axis=1)
p['robot_number'] = p.apply(robot_number, axis=1)
p['auto_community'] = p.apply(auto_community, axis=1)
p['auto_docked'] = p.apply(auto_docked, axis=1)
p['auto_docked_level'] = p.apply(auto_docked_level, axis=1)
p['auto_game_piece_points'] = p.apply(auto_gp_points, axis=1)
p['auto_points'] = p.apply(auto_points, axis=1)
p['tele_game_piece_points'] = p.apply(tele_gp_points, axis=1)
p['tele_points'] = p.apply(tele_points, axis=1)
p['endgame_points'] = p.apply(endgame_points, axis=1)
p['endgame_parked'] = p.apply(endgame_parked, axis=1)
p['endgame_docked'] = p.apply(endgame_docked, axis=1)
p['endgame_docked_level'] = p.apply(endgame_docked_level, axis=1)
p['link_points'] = p.apply(link_points, axis=1)
p['ranking_points'] = p.apply(rank_points, axis=1)
p['total_points'] = p.apply(total_points, axis=1)

In [25]:
cols = {
    'comp_level': 'string',
    'event_key': 'string',
    'key': 'string',
    'match_number': 'int16',
    'winning_alliance': 'string',
    'team_key': 'string',
    'alliance': 'string',
    'robot_number': 'int16',
    'auto_community': 'double',
    'auto_docked': 'double',
    'auto_docked_level': 'double',
    'auto_game_piece_points': 'double',
    'auto_points': 'double',
    'tele_game_piece_points': 'double',
    'tele_points': 'double',
    'endgame_points': 'double',
    'endgame_parked': 'double',
    'endgame_docked': 'double',
    'endgame_docked_level': 'double',
    'link_points': 'double',
    'ranking_points': 'double',
    'total_points': 'double'
}

fin = p[cols.keys()]
fin = fin.astype(cols)
# Remove any matches that have not yet been played
fin.dropna(subset='total_points', inplace=True)
fin.to_parquet('gvsu_data.parquet')
fin.to_csv('gvsu_data.csv')

In [26]:
import os
outpath = './graphs/'
if not os.path.exists(outpath):
    os.mkdir(outpath)

gd = fin.copy()

dropcols = [
    'comp_level',
    'event_key',
    'key',
    'match_number',
    'winning_alliance',
    'alliance',
    'robot_number'
]
gd.drop(columns=dropcols, inplace=True)
gd = gd.groupby('team_key').mean()
gd.to_parquet('gvsu_mean.parquet')
gd.to_csv('gvsu_mean.csv')